In [2]:
%load_ext autoreload
%autoreload 1

In [3]:
%aimport exv2.experiment_results 

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import glasbey



pd.set_option('display.max_columns', None)
sns.set_theme(rc={'figure.figsize':(12, 6)})
sns.set_context("paper")
sns.set_style("whitegrid")

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

palette = glasbey.create_block_palette(
    [4, 3, 3, 2, 2],
    colorblind_safe=True,
    cvd_severity=90
)
sns.set_palette(palette)

dep_focus = ["baseline_vanilla_full","monolith_feature_monolith_full","serverless_feature_serverless_full"]
ms_focus = ["baseline_vanilla_full","obs_feature_object-storage_full","norec_feature_norecommendations_full"]
lable_names = {
    "baseline_vanilla_full": "Microservice Baseline",
    'jvm_jvm-impoove_full': "Runtime Improvement",
    'monolith_feature_monolith_full': "Monolith Variant",
    'norec_feature_norecommendations_full' : "Service Reduction",
    'obs_feature_object-storage_full' : "Infrastructure Service Replacement",
    'serverless_feature_serverless_full' : "Serverless Variant",
}

In [4]:
exr = exv2.experiment_results.ExperimentResults("data/2024-07-*", load_stats_history=True) # type: ignore
RUN_VARS = exv2.experiment_results.ExperimentResults.RUN_VARS # type: ignore

assert("loadgenerator" not in (list(exr.pods['name'].unique())))
assert("unkown" not in (list(exr.pods['instance'].unique())))


/Users/b/projects/teastore_experiments/exv2/experiment_results.py:182: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_pods = pd.concat(


In [5]:

left = "exp_scale_pausing"
right = "exp_scale_rampup"


failures = exr.stats_aggregated[exr.stats_aggregated["exp_workload"].isin([left,right])].groupby(["exp_branch","exp_workload"])[["Request Count","Failure Count"]].sum()
failures["Failure Rate"] = failures["Failure Count"] / failures["Request Count"]
failures = failures.unstack()
failures["fr"] = failures["Failure Rate"].apply(lambda x: f'{x[left]:>2.2f} - {x[right]:>2.2f}', axis=1)
failures = failures.droplevel(1, axis=1).reset_index()[["exp_branch","fr"]]


In [6]:
main_table = failures[failures["exp_branch"].isin(dep_focus)].copy()
main_table["exp_branch"] = main_table["exp_branch"].map(lable_names)
main_table.rename(columns={"exp_branch":"Feature","fr":"Failure Rate [%] (idle-stress)"}, inplace=True)
main_table

with open("figs/failures_table_deployment.tex", "w") as f:
    f.write(main_table.to_latex(index=False, escape=False))

In [7]:
main_table = failures[failures["exp_branch"].isin(ms_focus)].copy()
main_table["exp_branch"] = main_table["exp_branch"].map(lable_names)
main_table.rename(columns={"exp_branch":"Feature","fr":"Failure Rate [%] (idle-stress)"}, inplace=True)
main_table

with open("figs/failures_table_ms_improv.tex", "w") as f:
    f.write(main_table.to_latex(index=False, escape=False))